In [2]:
import os
from dotenv import load_dotenv
 
load_dotenv()

TOGETHER_KEY = os.getenv('TOGETHER_API_KEY')
OPENAI_KEY = os.getenv('OPENAI_API_KEY')

In [58]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_together import ChatTogether
 
from typing import List
 
topic_identifier_system = """Analyze user input and identify the physics topic mentioned in the input. Do not return the input text.
For example:
- "Create questions about velocity" -> "velocity"
- "Explain displacement" -> "displacement"
 
Return only the identified physics topic name as given in the input."""
 
topic_check = ChatPromptTemplate.from_messages([
   ("system", topic_identifier_system),
   ("placeholder", "{messages}")
]) | ChatTogether(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo", temperature=0
    )
 
input = "Create 5 questions on energy."
 
topic = topic_check.invoke({"messages": [("user", input)]}).content

In [59]:
topic

'energy'

In [60]:
from langchain_core.prompts import ChatPromptTemplate

from langchain_together import ChatTogether
from langchain_openai import ChatOpenAI

import json

system_prompt = (
    "Create one multiple choice question for {skill} level of Bloom's taxonomy for a 9th grade Physics student in India on {topic}."
    "Requirements:"
    "Student should only be able to answer if they've mastered the concept"
    "Each distractor must address either: A specific misconception about {topic} or A prerequisite knowledge gap"
    "Language and complexity suitable for 9th grade"
    "Physics context and application"
    "Format as the output as a JSON:"
 
        """{{{{
        "question": "",
        "skill": ""
        "options": {{"a": "", "b": "", "c": "", "d": ""}},
        "correct": "",
        "explanation": {{
            "correct": "",
            "a": "misconception/prerequisite tested",
            "b": "", "c": "", "d": ""
        }}
        }}}}"""
 
    "For {skill} level, ensure:"
    "{skill_requirement}"
    "Make sure there are no additional information being other than the output in the format that is asked for."
)

skill_requirements = {
   "Remember": "Question tests ability to retrieve relevant knowledge from long-term memory.",
   "Understand": "Question tests ability to onstruct meaning from instructional messages, including oral, written, and graphic communication.",
   "Apply": "Question tests ability to carry out or use a procedure in a given situation.",
   "Analyze": "Question tests ability to break material into foundational parts and determine how parts relate to one another and the overall structure or purpose.",
   "Evaluate": "Question tests ability to make judgments based on criteria and standards."
}
 
skills = ["Remember", "Understand", "Apply", "Analyze", "Evaluate"]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# llm = ChatOpenAI(model="gpt-4o",
#                  api_key=os.getenv("OPENAI_API_KEY"))

# llm = ChatTogether(
#     model="Qwen/Qwen2.5-72B-Instruct-Turbo"
#     )

llm = ChatTogether(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo"
    )

# llm = ChatTogether(
#     model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo"
#     )

query = {"input": '{input}'}

def generate_assessment(llm):
   responses = {
       "topic": topic,
       "questions": []
   }
   
   for skill in skills:

       prompt = system_prompt.format(
           skill=skill,
           topic=topic,
           skill_requirement=skill_requirements[skill]
       )

       response = llm.invoke(prompt)
       cleaned_content = response.content.strip()
       if cleaned_content.startswith("```json"):
           cleaned_content = cleaned_content[7:-3]
           
       try:
           question_json = json.loads(cleaned_content)
           responses["questions"].append(question_json)
       except json.JSONDecodeError as e:
           print(response)
           print(f"Error parsing {skill} response")
   
   model_name = llm.model_name.split('/')[-1]
   filename = f"LLM_{topic}_{model_name}.json"
   
   with open(filename, "w") as f:
       json.dump(responses, f, indent=2, ensure_ascii=False)
       
   return responses
 
final_assessment = generate_assessment(llm)